# Limpieza de los datos

### Importación de Librerías y Configuración Inicial

In [28]:
import pandas as pd
import numpy as np
import warnings
import os
import re
from pathlib import Path
import json
from datetime import datetime

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

### Constantes para limpieza de datos

In [2]:
REPLACEMENT_MAPPING = {
    'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
    'ñ': 'n', 'ü': 'u', 'ç': 'c', 
    'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U',
    'Ñ': 'N', 'Ü': 'U', 'Ç': 'C',
    ',': '_', '-': '_', '/': '_', ' ': '_', '.': '_',
    '(': '', ')': '', '[': '', ']': '', '{': '', '}': '',
    'º': '', 'ª': ''
}

ARTICLES_MAPPING = {article: '_' for article in [
    '_el_', '_la_', '_los_', '_las_', '_un_', '_una_', '_unos_', '_unas_',
    '_del_', '_de_', '_y_', '_o_'
]}

### Constantes para sustitución de datos

Para diagnósticos:

In [3]:
CIE10_TRASTORNOS_MENTALES = {
    # F20-F29: Esquizofrenia, trastornos esquizotípicos y trastornos delirantes
    'F20.0': 'Esquizofrenia paranoide',
    'F20.1': 'Esquizofrenia hebefrenica',
    'F20.2': 'Esquizofrenia catatónica',
    'F20.3': 'Esquizofrenia indiferenciada',
    'F20.5': 'Esquizofrenia residual',
    'F20.81': 'Trastorno esquizofreniforme',
    'F20.89': 'Otras esquizofrenias',
    'F20.9': 'Esquizofrenia, no especificada',
    'F21': 'Trastorno esquizotípico',
    'F24': 'Trastorno psicótico compartido',
    'F25.0': 'Trastorno esquizoafectivo tipo bipolar',
    'F25.1': 'Trastorno esquizoafectivo tipo depresivo',
    'F25.8': 'Otros trastornos esquizoafectivos',
    'F25.9': 'Trastorno esquizoafectivo, no especificado',
    'F28': 'Otros trastornos psicóticos no orgánicos',
    'F29': 'Psicosis no orgánica, sin especificar',
    
    # F30-F39: Trastornos del humor (afectivos)
    'F30.2': 'Manía con síntomas psicóticos',
    'F30.8': 'Otros episodios maníacos',
    'F30.9': 'Episodio maníaco, no especificado',
    'F30.10': 'Episodio maníaco sin síntomas psicóticos, no especificado',
    'F30.11': 'Episodio maníaco sin síntomas psicóticos, leve',
    'F30.12': 'Episodio maníaco sin síntomas psicóticos, moderado',
    'F30.13': 'Episodio maníaco sin síntomas psicóticos, grave',
    'F31.0': 'Trastorno bipolar, episodio hipomaníaco actual',
    'F31.2': 'Trastorno bipolar, episodio maníaco grave con síntomas psicóticos',
    'F31.4': 'Trastorno bipolar, episodio depresivo grave con síntomas psicóticos',
    'F31.5': 'Trastorno bipolar, episodio depresivo grave sin síntomas psicóticos',
    'F31.9': 'Trastorno bipolar, no especificado',
    'F31.10': 'Trastorno bipolar, episodio maníaco leve actual',
    'F31.11': 'Trastorno bipolar, episodio maníaco moderado actual',
    'F31.12': 'Trastorno bipolar, episodio maníaco grave sin síntomas psicóticos',
    'F31.13': 'Trastorno bipolar, episodio maníaco grave con síntomas psicóticos',
    'F31.30': 'Trastorno bipolar, episodio depresivo leve o moderado, no especificado',
    'F31.31': 'Trastorno bipolar, episodio depresivo leve',
    'F31.32': 'Trastorno bipolar, episodio depresivo moderado',
    'F31.60': 'Trastorno bipolar, episodio mixto, no especificado',
    'F31.61': 'Trastorno bipolar, episodio mixto leve',
    'F31.62': 'Trastorno bipolar, episodio mixto moderado',
    'F31.63': 'Trastorno bipolar, episodio mixto grave sin síntomas psicóticos',
    'F31.64': 'Trastorno bipolar, episodio mixto grave con síntomas psicóticos',
    'F31.70': 'Trastorno bipolar en remisión parcial, episodio más reciente no especificado',
    'F31.73': 'Trastorno bipolar en remisión parcial, episodio más reciente maníaco',
    'F31.74': 'Trastorno bipolar en remisión parcial, episodio más reciente mixto',
    'F31.75': 'Trastorno bipolar en remisión parcial, episodio más reciente depresivo',
    'F31.81': 'Trastorno bipolar II',
    'F31.89': 'Otros trastornos bipolares',
    'F32.0': 'Episodio depresivo leve',
    'F32.1': 'Episodio depresivo moderado',
    'F32.2': 'Episodio depresivo grave sin síntomas psicóticos',
    'F32.3': 'Episodio depresivo grave con síntomas psicóticos',
    'F32.89': 'Otros episodios depresivos',
    'F32.9': 'Episodio depresivo, no especificado',
    'F33.0': 'Trastorno depresivo recurrente, episodio leve actual',
    'F33.1': 'Trastorno depresivo recurrente, episodio moderado actual',
    'F33.2': 'Trastorno depresivo recurrente, episodio grave sin síntomas psicóticos',
    'F33.3': 'Trastorno depresivo recurrente, episodio grave con síntomas psicóticos',
    'F33.8': 'Otros trastornos depresivos recurrentes',
    'F33.9': 'Trastorno depresivo recurrente, no especificado',
    'F33.40': 'Trastorno depresivo recurrente en remisión, no especificado',
    'F33.41': 'Trastorno depresivo recurrente en remisión parcial',
    'F33.42': 'Trastorno depresivo recurrente en remisión completa',
    'F34.0': 'Ciclotimia',
    'F34.1': 'Distimia',
    'F34.89': 'Otros trastornos persistentes del estado de ánimo',
    'F34.9': 'Trastorno persistente del estado de ánimo, no especificado',
    'F39': 'Trastorno del humor no especificado',
    
    # F40-F48: Trastornos neuróticos, relacionados con estrés y somatomorfos
    'F40.00': 'Agorafobia sin trastorno de pánico, no especificado',
    'F40.01': 'Agorafobia con trastorno de pánico',
    'F40.02': 'Agorafobia sin trastorno de pánico',
    'F40.8': 'Otras fobias específicas',
    'F40.9': 'Trastorno fóbico de ansiedad, no especificado',
    'F40.10': 'Fobia social, no especificada',
    'F40.298': 'Otras fobias específicas',
    'F41.0': 'Trastorno de pánico',
    'F41.1': 'Trastorno de ansiedad generalizada',
    'F41.3': 'Otros trastornos de ansiedad mixtos',
    'F41.8': 'Otros trastornos de ansiedad especificados',
    'F41.9': 'Trastorno de ansiedad, no especificado',
    'F42.2': 'Trastorno obsesivo-compulsivo mixto',
    'F42.8': 'Otros trastornos obsesivo-compulsivos',
    'F42.9': 'Trastorno obsesivo-compulsivo, no especificado',
    'F43.0': 'Reacción aguda al estrés',
    'F43.10': 'Trastorno de estrés postraumático, no especificado',
    'F43.11': 'Trastorno de estrés postraumático, agudo',
    'F43.12': 'Trastorno de estrés postraumático, crónico',
    'F43.21': 'Trastorno de adaptación con estado de ánimo deprimido',
    'F43.22': 'Trastorno de adaptación con ansiedad',
    'F43.23': 'Trastorno de adaptación con ansiedad y estado de ánimo deprimido mixtos',
    'F43.24': 'Trastorno de adaptación con alteración de la conducta',
    'F43.25': 'Trastorno de adaptación con alteración mixta de las emociones y la conducta',
    'F43.29': 'Trastorno de adaptación con otros síntomas',
    'F43.8': 'Otras reacciones al estrés grave',
    'F44.0': 'Amnesia disociativa',
    'F44.1': 'Fuga disociativa',
    'F44.4': 'Trastornos disociativos de la motricidad',
    'F44.5': 'Convulsiones disociativas',
    'F44.6': 'Anestesia y pérdida sensorial disociativas',
    'F44.7': 'Trastorno disociativo mixto',
    'F44.81': 'Trastorno de identidad disociativo',
    'F44.89': 'Otros trastornos disociativos',
    'F44.9': 'Trastorno disociativo, no especificado',
    'F45.0': 'Trastorno de somatización',
    'F45.1': 'Trastorno somatomorfo indiferenciado',
    'F45.8': 'Otros trastornos somatomorfos',
    'F45.9': 'Trastorno somatomorfo, no especificado',
    'F45.20': 'Trastorno de ansiedad por enfermedad, no especificado',
    'F45.21': 'Hipocondría',
    'F45.22': 'Trastorno dismórfico corporal',
    'F48.1': 'Trastorno de despersonalización-desrealización',
    'F48.8': 'Otros trastornos neuróticos especificados',
    
    # F50-F59: Síndromes del comportamiento
    'F50.00': 'Anorexia nerviosa, no especificada',
    'F50.01': 'Anorexia nerviosa, tipo restrictivo',
    'F50.02': 'Anorexia nerviosa, tipo con atracones/purgas',
    'F50.2': 'Bulimia nerviosa',
    'F50.89': 'Otros trastornos de la conducta alimentaria',
    'F50.9': 'Trastorno de la conducta alimentaria, no especificado',
    'F51.3': 'Sonambulismo',
    'F51.8': 'Otros trastornos del sueño no orgánicos',
    'F51.11': 'Trastorno de insomnio primario',
    'F51.19': 'Otros trastornos de insomnio',
    
    # F60-F69: Trastornos de la personalidad
    'F60.0': 'Trastorno de personalidad paranoide',
    'F60.1': 'Trastorno de personalidad esquizoide',
    'F60.2': 'Trastorno de personalidad antisocial',
    'F60.3': 'Trastorno de personalidad emocionalmente inestable',
    'F60.4': 'Trastorno de personalidad histriónico',
    'F60.5': 'Trastorno de personalidad anancástico',
    'F60.6': 'Trastorno de personalidad por evitación',
    'F60.7': 'Trastorno de personalidad dependiente',
    'F60.81': 'Trastorno de personalidad narcisista',
    'F60.89': 'Otros trastornos de personalidad específicos',
    'F60.9': 'Trastorno de personalidad, no especificado',
    'F63.0': 'Ludopatía',
    'F63.2': 'Cleptomanía',
    'F63.81': 'Trastorno explosivo intermitente',
    'F63.89': 'Otros trastornos del control de impulsos',
    'F63.9': 'Trastorno del control de impulsos, no especificado',
    'F64.1': 'Transexualismo',
    'F68.8': 'Otros trastornos especificados de la personalidad y del comportamiento del adulto',
    'F68.10': 'Trastorno facticio, no especificado',
    'F68.11': 'Trastorno facticio con síntomas predominantemente psicológicos',
    'F68.12': 'Trastorno facticio con síntomas físicos predominantes',
    'F69': 'Trastorno de la personalidad y del comportamiento del adulto, no especificado',
    
    # F10-F19: Trastornos mentales y del comportamiento debidos al consumo de sustancias
    'F10.10': 'Trastorno por consumo de alcohol, leve',
    'F10.14': 'Trastorno por consumo de alcohol, leve, con trastorno bipolar inducido',
    'F10.19': 'Trastorno por consumo de alcohol, leve, con trastorno no especificado',
    'F10.120': 'Trastorno por consumo de alcohol, leve, con intoxicación no complicada',
    'F10.121': 'Trastorno por consumo de alcohol, leve, con intoxicación con delirium',
    'F10.129': 'Trastorno por consumo de alcohol, leve, con intoxicación no especificada',
    'F10.150': 'Trastorno por consumo de alcohol, leve, con trastorno psicótico inducido',
    'F10.151': 'Trastorno por consumo de alcohol, leve, con trastorno psicótico con delirios',
    'F10.159': 'Trastorno por consumo de alcohol, leve, con trastorno psicótico no especificado',
    'F10.180': 'Trastorno por consumo de alcohol, leve, con trastorno de ansiedad inducido',
    'F10.188': 'Trastorno por consumo de alcohol, leve, con otros trastornos inducidos',
    'F10.20': 'Trastorno por consumo de alcohol, moderado',
    'F10.21': 'Trastorno por consumo de alcohol, moderado, en remisión temprana',
    'F10.220': 'Trastorno por consumo de alcohol, moderado, con intoxicación no complicada',
    'F10.221': 'Trastorno por consumo de alcohol, moderado, con intoxicación con delirium',
    'F10.229': 'Trastorno por consumo de alcohol, moderado, con intoxicación no especificada',
    'F10.230': 'Trastorno por consumo de alcohol, moderado, con abstinencia no complicada',
    'F10.231': 'Trastorno por consumo de alcohol, moderado, con abstinencia con delirium',
    'F10.232': 'Trastorno por consumo de alcohol, moderado, con abstinencia con alteraciones perceptivas',
    'F10.239': 'Trastorno por consumo de alcohol, moderado, con abstinencia no especificada',
    'F10.24': 'Trastorno por consumo de alcohol, moderado, con trastorno del estado de ánimo inducido',
    'F10.250': 'Trastorno por consumo de alcohol, moderado, con trastorno psicótico inducido con delirios',
    'F10.251': 'Trastorno por consumo de alcohol, moderado, con trastorno psicótico inducido con alucinaciones',
    'F10.259': 'Trastorno por consumo de alcohol, moderado, con trastorno psicótico inducido no especificado',
    'F10.26': 'Trastorno por consumo de alcohol, moderado, con trastorno amnésico persistente inducido',
    'F10.27': 'Trastorno por consumo de alcohol, moderado, con demencia persistente inducida',
    'F10.280': 'Trastorno por consumo de alcohol, moderado, con trastorno de ansiedad inducido',
    'F10.288': 'Trastorno por consumo de alcohol, moderado, con otros trastornos inducidos',
    'F10.29': 'Trastorno por consumo de alcohol, moderado, con trastorno no especificado inducido',
    'F10.921': 'Uso de alcohol con intoxicación con delirium',
    'F10.929': 'Uso de alcohol con intoxicación no especificada',
    'F10.94': 'Uso de alcohol con trastorno del estado de ánimo inducido',
    'F10.950': 'Uso de alcohol con trastorno psicótico inducido con delirios',
    'F10.951': 'Uso de alcohol con trastorno psicótico inducido con alucinaciones',
    'F10.988': 'Uso de alcohol con otros trastornos inducidos',
    'F10.99': 'Uso de alcohol con trastorno no especificado',
    'F11.20': 'Trastorno por consumo de opioides, moderado',
    'F11.21': 'Trastorno por consumo de opioides, moderado, en remisión temprana',
    'F11.23': 'Trastorno por consumo de opioides, moderado, con abstinencia',
    'F11.229': 'Trastorno por consumo de opioides, moderado, con intoxicación no especificada',
    'F11.250': 'Trastorno por consumo de opioides, moderado, con trastorno psicótico inducido con delirios',
    'F11.251': 'Trastorno por consumo de opioides, moderado, con trastorno psicótico inducido con alucinaciones',
    'F11.259': 'Trastorno por consumo de opioides, moderado, con trastorno psicótico inducido no especificado',
    'F11.288': 'Trastorno por consumo de opioides, moderado, con otros trastornos inducidos',
    'F11.93': 'Uso de opioides con abstinencia',
    'F11.94': 'Uso de opioides con trastorno del estado de ánimo inducido',
    'F12.20': 'Trastorno por consumo de cannabis, moderado',
    'F12.21': 'Trastorno por consumo de cannabis, moderado, en remisión temprana',
    'F12.221': 'Trastorno por consumo de cannabis, moderado, con intoxicación con delirium',
    'F12.229': 'Trastorno por consumo de cannabis, moderado, con intoxicación no especificada',
    'F12.250': 'Trastorno por consumo de cannabis, moderado, con trastorno psicótico inducido con delirios',
    'F12.251': 'Trastorno por consumo de cannabis, moderado, con trastorno psicótico inducido con alucinaciones',
    'F12.259': 'Trastorno por consumo de cannabis, moderado, con trastorno psicótico inducido no especificado',
    'F12.280': 'Trastorno por consumo de cannabis, moderado, con trastorno de ansiedad inducido',
    'F12.288': 'Trastorno por consumo de cannabis, moderado, con otros trastornos inducidos',
    'F12.29': 'Trastorno por consumo de cannabis, moderado, con trastorno no especificado inducido',
    'F12.950': 'Uso de cannabis con trastorno psicótico inducido con delirios',
    'F12.959': 'Uso de cannabis con trastorno psicótico inducido no especificado',
    'F12.988': 'Uso de cannabis con otros trastornos inducidos',
    'F12.99': 'Uso de cannabis con trastorno no especificado',
    'F13.20': 'Trastorno por consumo de sedantes, hipnóticos o ansiolíticos, moderado',
    'F13.220': 'Trastorno por consumo de sedantes, moderado, con intoxicación no complicada',
    'F13.230': 'Trastorno por consumo de sedantes, moderado, con abstinencia no complicada',
    'F13.231': 'Trastorno por consumo de sedantes, moderado, con abstinencia con delirium',
    'F13.259': 'Trastorno por consumo de sedantes, moderado, con trastorno psicótico inducido no especificado',
    'F13.280': 'Trastorno por consumo de sedantes, moderado, con trastorno de ansiedad inducido',
    'F13.288': 'Trastorno por consumo de sedantes, moderado, con otros trastornos inducidos',
    'F13.29': 'Trastorno por consumo de sedantes, moderado, con trastorno no especificado inducido',
    'F13.931': 'Uso de sedantes con abstinencia con delirium',
    'F13.939': 'Uso de sedantes con abstinencia no especificada',
    'F13.959': 'Uso de sedantes con trastorno psicótico inducido no especificado',
    'F13.982': 'Uso de sedantes con trastorno del sueño inducido',
    'F14.20': 'Trastorno por consumo de cocaína, moderado',
    'F14.21': 'Trastorno por consumo de cocaína, moderado, en remisión temprana',
    'F14.220': 'Trastorno por consumo de cocaína, moderado, con intoxicación no complicada',
    'F14.221': 'Trastorno por consumo de cocaína, moderado, con intoxicación con delirium',
    'F14.222': 'Trastorno por consumo de cocaína, moderado, con intoxicación con alteraciones perceptivas',
    'F14.229': 'Trastorno por consumo de cocaína, moderado, con intoxicación no especificada',
    'F14.23': 'Trastorno por consumo de cocaína, moderado, con abstinencia',
    'F14.24': 'Trastorno por consumo de cocaína, moderado, con trastorno del estado de ánimo inducido',
    'F14.250': 'Trastorno por consumo de cocaína, moderado, con trastorno psicótico inducido con delirios',
    'F14.251': 'Trastorno por consumo de cocaína, moderado, con trastorno psicótico inducido con alucinaciones',
    'F14.259': 'Trastorno por consumo de cocaína, moderado, con trastorno psicótico inducido no especificado',
    'F14.280': 'Trastorno por consumo de cocaína, moderado, con trastorno de ansiedad inducido',
    'F14.288': 'Trastorno por consumo de cocaína, moderado, con otros trastornos inducidos',
    'F14.29': 'Trastorno por consumo de cocaína, moderado, con trastorno no especificado inducido',
    'F14.90': 'Uso de cocaína, no complicado',
    'F14.94': 'Uso de cocaína con trastorno del estado de ánimo inducido',
    'F14.950': 'Uso de cocaína con trastorno psicótico inducido con delirios',
    'F14.959': 'Uso de cocaína con trastorno psicótico inducido no especificado',
    'F14.988': 'Uso de cocaína con otros trastornos inducidos',
    'F18.20': 'Trastorno por consumo de inhalantes, moderado',
    'F19.20': 'Trastorno por consumo de otras sustancias psicoactivas, moderado',
    'F19.24': 'Trastorno por consumo de otras sustancias, moderado, con trastorno del estado de ánimo inducido',
    'F19.220': 'Trastorno por consumo de otras sustancias, moderado, con intoxicación no complicada',
    'F19.221': 'Trastorno por consumo de otras sustancias, moderado, con intoxicación con delirium',
    'F19.229': 'Trastorno por consumo de otras sustancias, moderado, con intoxicación no especificada',
    'F19.250': 'Trastorno por consumo de otras sustancias, moderado, con trastorno psicótico inducido con delirios',
    'F19.251': 'Trastorno por consumo de otras sustancias, moderado, con trastorno psicótico inducido con alucinaciones',
    'F19.259': 'Trastorno por consumo de otras sustancias, moderado, con trastorno psicótico inducido no especificado',
    'F19.29': 'Trastorno por consumo de otras sustancias, moderado, con trastorno no especificado inducido',
    'F19.288': 'Trastorno por consumo de otras sustancias, moderado, con otros trastornos inducidos',
    'F19.922': 'Uso de otras sustancias con intoxicación con alteraciones perceptivas',
    'F19.950': 'Uso de otras sustancias con trastorno psicótico inducido con delirios',
    'F19.959': 'Uso de otras sustancias con trastorno psicótico inducido no especificado',
    'F19.97': 'Uso de otras sustancias con demencia persistente inducida',
    'F19.988': 'Uso de otras sustancias con otros trastornos inducidos',
    'F19.99': 'Uso de otras sustancias con trastorno no especificado',
    
    # F90-F98: Trastornos del comportamiento y de las emociones de comienzo habitual en la infancia y adolescencia
    'F91.0': 'Trastorno disocial limitado al contexto familiar',
    'F91.8': 'Otros trastornos disociales',
    'F91.9': 'Trastorno disocial, no especificado',
    'F93.0': 'Trastorno de ansiedad de separación de la infancia',
    'F93.8': 'Otros trastornos de las emociones de la infancia',
    'F94.0': 'Mutismo selectivo',
    'F94.8': 'Otros trastornos del comportamiento social de la infancia',
    'F95.0': 'Trastorno de tics transitorio',
    'F95.1': 'Trastorno de tics motor o vocal crónico',
    'F95.2': 'Trastorno de tics múltiples motores y vocales (Tourette)',
    'F95.8': 'Otros trastornos de tics',
    'F95.9': 'Trastorno de tics, no especificado',
    'F99': 'Trastorno mental, no especificado'
}

CIE10_CODIGOS_ADICIONALES = {
    # Códigos Z: Factores que influyen en el estado de salud
    'Z63.79': 'Otros problemas especificados relacionados con el grupo primario de apoyo',
    'Z62.820': 'Padre o madre en servicio militar',
    'Z91.19': 'Historia personal de incumplimiento de otros tratamientos médicos',
    'Z72.0': 'Uso de tabaco',
    'Z95.0': 'Presencia de marcapasos cardíaco',
    'Z88.8': 'Alergia a otros fármacos, medicamentos y sustancias biológicas',
    'Z88.0': 'Alergia a la penicilina',
    'Z88.2': 'Alergia a sulfonamidas',
    'Z91.14': 'Historia personal de alergia a anestésicos',
    'Z63.9': 'Problema relacionado con el grupo primario de apoyo, no especificado',
    'Z79.82': 'Uso prolongado de aspirina',
    'Z79.01': 'Uso prolongado de anticoagulantes',
    'Z79.3': 'Uso prolongado de hormonas e insulina',
    'Z79.4': 'Uso prolongado de insulina',
    'Z79.891': 'Uso prolongado de agentes antiinflamatorios no esteroides',
    'Z79.899': 'Otros usos prolongados de medicamentos',
    'Z86.73': 'Historia personal de enfermedad cerebrovascular transitoria sin secuelas residuales',
    'Z86.718': 'Historia personal de otros trastornos del sistema circulatorio',
    'Z81.8': 'Historia familiar de otros trastornos mentales y del comportamiento',
    'Z81.1': 'Historia familiar de abuso de alcohol',
    'Z91.81': 'Historia de incumplimiento de tratamiento médico',
    'Z91.5': 'Historia personal de autolesión',
    'Z91.041': 'Historia personal de alergia alimentaria a mariscos',
    'Z56.0': 'Desempleo, no especificado',
    'Z60.2': 'Problemas relacionados con vivir solo',
    'Z60.8': 'Otros problemas relacionados con el ambiente social',
    'Z90.710': 'Ausencia adquirida de ambas mamas',
    'Z90.49': 'Ausencia adquirida de otra parte del tracto digestivo',
    'Z99.89': 'Dependencia de otras máquinas y dispositivos facilitadores',
    'Z99.81': 'Dependencia de oxígeno suplementario',
    'Z96.1': 'Presencia de implante intraocular de lente',
    'Z98.42': 'Historia de mamoplastia',
    'Z85.46': 'Historia personal de neoplasia maligna de próstata',
    'Z87.891': 'Historia personal de infección por nicotina',
    
    # Códigos I: Enfermedades del sistema circulatorio
    'I10': 'Hipertensión esencial (primaria)',
    'I11.9': 'Enfermedad cardíaca hipertensiva sin insuficiencia cardíaca',
    'I35.8': 'Otros trastornos de la válvula aórtica',
    'I42.0': 'Cardiomiopatía dilatada',
    'I42.2': 'Otra cardiomiopatía hipertrófica',
    'I48.2': 'Fibrilación auricular crónica',
    'I50.1': 'Insuficiencia ventricular izquierda',
    'I87.2': 'Insuficiencia venosa (crónica) (periférica)',
    'I83.009': 'Venas varicosas de la extremidad inferior derecha con úlcera no especificada',
    'I25.10': 'Enfermedad cardíaca arterioesclerótica de arteria coronaria nativa sin angina pectoris',
    'I71.9': 'Aneurisma aórtico de localización no especificada',
    
    # Códigos E: Enfermedades endocrinas, nutricionales y metabólicas
    'E66.9': 'Obesidad, no especificada',
    'E66.01': 'Obesidad mórbida debida a exceso de calorías',
    'E66.8': 'Otras obesidades',
    'E79.0': 'Hiperuricemia sin signos de artritis inflamatoria y enfermedad tofácea',
    'E11.9': 'Diabetes mellitus tipo 2 sin complicaciones',
    'E11.319': 'Diabetes mellitus tipo 2 con retinopatía diabética no especificada sin edema macular',
    'E87.6': 'Hipokalemia',
    'E78.5': 'Hiperlipidemia, no especificada',
    'E78.0': 'Hipercolesterolemia pura',
    'E03.9': 'Hipotiroidismo, no especificado',
    'E04.2': 'Bocio multinodular no tóxico',
    'E21.0': 'Hiperparatiroidismo primario',
    'E02': 'Hipotiroidismo subclínico por deficiencia de yodo',
    'E46': 'Desnutrición proteicocalórica, no especificada',
    'E53.9': 'Deficiencia de vitamina B, no especificada',
    
    # Códigos F adicionales (trastornos mentales no incluidos antes)
    'F17.210': 'Dependencia de nicotina, cigarrillos, no complicada',
    'F17.200': 'Dependencia de nicotina, producto de tabaco no especificado, no complicada',
    'F79': 'Retraso mental, grado no especificado',
    'F12.10': 'Trastorno por consumo de cannabis, leve',
    'F14.10': 'Trastorno por consumo de cocaína, leve',
    'F14.188': 'Trastorno por consumo de cocaína, leve, con otros trastornos inducidos',
    'F01.51': 'Demencia vascular con alteración del comportamiento',
    'F05': 'Delirium debido a condición médica conocida',
    'F15.10': 'Trastorno por consumo de otros estimulantes, leve',
    'F11.10': 'Trastorno por consumo de opioides, leve',
    'F32.8': 'Otros episodios depresivos',
    
    # Códigos B: Enfermedades infecciosas y parasitarias
    'B20': 'Enfermedad por virus de la inmunodeficiencia humana [VIH]',
    'B18.2': 'Hepatitis viral crónica C',
    'B18.1': 'Hepatitis viral crónica B sin agente delta',
    'B19.20': 'Hepatitis viral C sin coma hepático',
    'B36.9': 'Micosis superficial, no especificada',
    'B96.89': 'Otros agentes bacterianos especificados como causa de enfermedades',
    
    # Códigos K: Enfermedades del sistema digestivo
    'K44.9': 'Hernia diafragmática sin obstrucción o gangrena',
    'K21.9': 'Enfermedad por reflujo gastroesofágico sin esofagitis',
    'K58.9': 'Síndrome del intestino irritable sin diarrea',
    'K57.30': 'Enfermedad diverticular del intestino grueso sin perforación o absceso sin sangrado',
    'K57.00': 'Enfermedad diverticular del intestino delgado con perforación y absceso sin sangrado',
    'K76.1': 'Congestión hepática pasiva crónica',
    'K80.20': 'Cálculos de la vesícula biliar sin colecistitis sin obstrucción',
    'K70.0': 'Hígado graso alcohólico',
    
    # Códigos R: Síntomas, signos y hallazgos anormales
    'R45.851': 'Ideación suicida',
    'R47.1': 'Disartria y anartria',
    'R19.7': 'Diarrea, no especificada',
    'R78.81': 'Hallazgo bacteriano en sangre',
    'R76.11': 'Título de anticuerpos antinucleares [ANA] no específico',
    
    # Códigos M: Enfermedades del sistema musculoesquelético
    'M76.50': 'Tendinitis rotuliana, rodilla no especificada',
    'M79.7': 'Fibromialgia',
    'M41.9': 'Escoliosis, no especificada',
    'M19.90': 'Artrosis, sitio no especificado',
    'M06.9': 'Artritis reumatoide, no especificada',
    'M35.3': 'Polimialgia reumática',
    
    # Códigos N: Enfermedades del sistema genitourinario
    'N92.1': 'Menstruación excesiva y frecuente con ciclo irregular',
    'N40.0': 'Hiperplasia prostática benigna sin síntomas del tracto urinario inferior',
    'N13.30': 'Hidronefrosis con infección del tracto urinario, no especificada',
    'N28.1': 'Quiste del riñón, adquirido',
    
    # Códigos J: Enfermedades del sistema respiratorio
    'J96.90': 'Insuficiencia respiratoria, no especificada',
    'J47.9': 'Bronquiectasia, no complicada',
    'J45.909': 'Asma, no especificada, no complicada',
    'J44.9': 'Enfermedad pulmonar obstructiva crónica, no especificada',
    
    # Códigos C: Neoplasias
    'C07': 'Neoplasia maligna de glándula parótida',
    
    # Códigos D: Enfermedades de la sangre
    'D75.89': 'Otras enfermedades especificadas de la sangre y órganos hematopoyéticos',
    'D25.9': 'Leiomioma del útero, no especificado',
    'D50.9': 'Anemia por deficiencia de hierro, no especificada',
    
    # Códigos G: Enfermedades del sistema nervioso
    'G21.11': 'Parkinsonismo inducido por neurolépticos',
    'G47.33': 'Apnea obstructiva del sueño',
    
    # Códigos H: Enfermedades del ojo y del oído
    'H54.0': 'Ceguera, ambos ojos',
    'H54.40': 'Ceguera, un ojo, visión baja otro ojo, no especificado',
    
    # Códigos Q: Malformaciones congénitas
    'Q61.3': 'Enfermedad poliquística del riñón, no especificada',
    
    # Códigos T: Lesiones, envenenamientos
    'T43.595S': 'Efecto adverso de otros antipsicóticos y neurolépticos, secuela',
    'T36.8X5A': 'Efecto adverso de otros antibióticos sistémicos, encuentro inicial',
    'S29.9XXA': 'Lesión no especificada del tórax, encuentro inicial',
    
    # Códigos X: Causas externas de morbilidad
    'X58.XXXA': 'Exposición a otros factores especificados, encuentro inicial',
    
    # Códigos A: Ciertas enfermedades infecciosas y parasitarias
    'A67.1': 'Lesiones de frambesía múltiples'
}

# Combinar ambos diccionarios
CIE10_COMPLETO = {**CIE10_TRASTORNOS_MENTALES, **CIE10_CODIGOS_ADICIONALES}

Para procedimientos:

In [24]:
PROCEDIMIENTOS_ICD10_PCS = {
    # Sección 0: Medical and Surgical (primeros caracteres)
    # 00: Central Nervous System and Cranial Nerves
    # 009: Drainage of Central Nervous System and Cranial Nerves
    '009500Z': 'Drenaje de espacio subaracnoideo lumbar, abordaje abierto',
    '009U30Z': 'Drenaje de médula espinal, abordaje percutáneo',
    '009U3ZX': 'Drenaje de médula espinal, abordaje percutáneo, diagnóstico',
    '009U3ZZ': 'Drenaje de médula espinal, abordaje percutáneo, sin dispositivo',
    '009Y3ZX': 'Drenaje de médula espinal y nervio espinal, percutáneo, diagnóstico',
    '009Y3ZZ': 'Drenaje de médula espinal y nervio espinal, percutáneo',
    
    # 00H: Insertion - Central Nervous System
    '00H03MZ': 'Inserción de estimulador en cerebro, abordaje percutáneo',
    '00HU3MZ': 'Inserción de estimulador en médula espinal, abordaje percutáneo',
    
    # 00N: Release - Central Nervous System
    '00NK3ZZ': 'Liberación de nervio craneal, abordaje percutáneo',
    
    # 02: Heart and Great Vessels
    '02583ZZ': 'Destrucción de conducción cardíaca, abordaje percutáneo',
    '02703DZ': 'Dilatación de arteria coronaria izquierda, dispositivo intraluminal',
    '02HK30Z': 'Inserción de desfibrilador en ventrículo derecho, percutáneo',
    '02HV33Z': 'Inserción de dispositivo de infusión en vena cava superior, percutáneo',
    '02WAXQZ': 'Revisión de marcapasos cardíaco, abordaje externo',
    
    # 03: Upper Arteries
    '031809D': 'Bypass de arteria vertebral derecha con autólogo arterial',
    
    # 04: Upper Veins
    '04L20ZZ': 'Oclusión de vena yugular interna derecha, abordaje abierto',
    
    # 05: Lower Veins
    '059Y3ZX': 'Drenaje de venas pélvicas, abordaje percutáneo, diagnóstico',
    '05H533Z': 'Inserción de dispositivo de infusión en vena cava inferior, percutáneo',
    '05HM33Z': 'Inserción de dispositivo de infusión en vena renal derecha, percutáneo',
    '05HP33Z': 'Inserción de dispositivo de infusión en vena esplénica, percutáneo',
}

Para países:

In [30]:
# Diccionario de códigos numéricos de países (normalizado)
PAISES_NUMERICOS = {
    '004': 'afganistan',
    '096': 'brunei',
    '248': 'aland',
    '100': 'bulgaria',
    '008': 'albania',
    '854': 'burkina faso',
    '276': 'alemania',
    '108': 'burundi',
    '020': 'andorra',
    '064': 'butan',
    '024': 'angola',
    '132': 'cabo verde',
    '660': 'anguila',
    '116': 'camboya',
    '028': 'antigua y barbuda',
    '120': 'camerun',
    '682': 'arabia saudita',
    '124': 'canada',
    '012': 'argelia',
    '634': 'catar',
    '032': 'argentina',
    '535': 'caribe neerlandes',
    '051': 'armenia',
    '148': 'chad',
    '533': 'aruba',
    '152': 'chile',
    '036': 'australia',
    '156': 'china',
    '040': 'austria',
    '196': 'chipre',
    '031': 'azerbaiyan',
    '170': 'colombia',
    '044': 'bahamas',
    '174': 'comoras',
    '050': 'banglades',
    '408': 'corea del norte',
    '052': 'barbados',
    '410': 'corea del sur',
    '048': 'bahrein',
    '384': 'costa de marfil',
    '056': 'belgica',
    '188': 'costa rica',
    '084': 'belice',
    '191': 'croacia',
    '204': 'benin',
    '192': 'cuba',
    '060': 'bermudas',
    '531': 'curazao',
    '112': 'bielorrusia',
    '208': 'dinamarca',
    '104': 'birmania',
    '212': 'dominica',
    '068': 'bolivia',
    '218': 'ecuador',
    '070': 'bosnia y herzegovina',
    '818': 'egipto',
    '072': 'botsuana',
    '222': 'el salvador',
    '076': 'brasil',
    '784': 'emiratos arabes unidos',
    '232': 'eritrea',
    '344': 'hong kong',
    '703': 'eslovaquia',
    '348': 'hungria',
    '705': 'eslovenia',
    '356': 'india',
    '724': 'espana',
    '360': 'indonesia',
    '840': 'estados unidos',
    '368': 'irak',
    '233': 'estonia',
    '364': 'iran',
    '231': 'etiopia',
    '372': 'irlanda',
    '608': 'filipinas',
    '833': 'isla de man',
    '246': 'finlandia',
    '574': 'norfolk',
    '242': 'fiyi',
    '352': 'islandia',
    '250': 'francia',
    '136': 'islas caiman',
    '266': 'gabon',
    '184': 'islas cook',
    '270': 'gambia',
    '234': 'islas feroe',
    '268': 'georgia',
    '238': 'islas malvinas',
    '288': 'ghana',
    '580': 'islas marianas del norte',
    '292': 'gibraltar',
    '584': 'islas marshall',
    '308': 'granada',
    '612': 'islas pitcairn',
    '300': 'grecia',
    '090': 'islas salomon',
    '304': 'groenlandia',
    '796': 'islas turcas y caicos',
    '312': 'guadalupe',
    '581': 'islas ultramarinas de estados unidos',
    '316': 'guam',
    '092': 'islas virgenes britanicas',
    '320': 'guatemala',
    '850': 'islas virgenes de los estados unidos',
    '254': 'guayana francesa',
    '376': 'israel',
    '831': 'guernsey',
    '380': 'italia',
    '324': 'guinea',
    '388': 'jamaica',
    '624': 'guinea-bisau',
    '392': 'japon',
    '226': 'guinea ecuatorial',
    '832': 'jersey',
    '328': 'guyana',
    '400': 'jordania',
    '332': 'haiti',
    '398': 'kazajistan',
    '340': 'honduras',
    '404': 'kenia',
    '417': 'kirguistan',
    '500': 'montserrat',
    '296': 'kiribati',
    '508': 'mozambique',
    '414': 'kuwait',
    '516': 'namibia',
    '418': 'laos',
    '520': 'nauru',
    '426': 'lesoto',
    '524': 'nepal',
    '428': 'letonia',
    '558': 'nicaragua',
    '422': 'libano',
    '562': 'niger',
    '430': 'liberia',
    '566': 'nigeria',
    '434': 'libia',
    '570': 'niue',
    '438': 'liechtenstein',
    '578': 'noruega',
    '440': 'lituania',
    '540': 'nueva caledonia',
    '442': 'luxemburgo',
    '554': 'nueva zelanda',
    '446': 'macao',
    '512': 'oman',
    '450': 'madagascar',
    '528': 'paises bajos',
    '458': 'malasia',
    '586': 'pakistan',
    '454': 'malaui',
    '585': 'palaos',
    '462': 'maldivas',
    '275': 'estado de palestina',
    '466': 'mali',
    '591': 'panama',
    '470': 'malta',
    '598': 'papua nueva guinea',
    '504': 'marruecos',
    '600': 'paraguay',
    '474': 'martinica',
    '604': 'peru',
    '480': 'mauricio',
    '258': 'polinesia francesa',
    '478': 'mauritania',
    '616': 'polonia',
    '175': 'mayotte',
    '620': 'portugal',
    '484': 'mexico',
    '630': 'puerto rico',
    '583': 'micronesia',
    '826': 'reino unido',
    '498': 'moldavia',
    '140': 'republica centroafricana',
    '492': 'monaco',
    '203': 'republica checa',
    '496': 'mongolia',
    '807': 'republica de macedonia',
    '499': 'montenegro',
    '178': 'republica del congo',
    '180': 'republica democratica del congo',
    '728': 'sudan del sur',
    '214': 'republica dominicana',
    '752': 'suecia',
    '638': 'reunion',
    '756': 'suiza',
    '646': 'ruanda',
    '740': 'surinam',
    '642': 'rumania',
    '744': 'svalbard y jan mayen',
    '643': 'rusia',
    '764': 'tailandia',
    '732': 'sahara occidental',
    '834': 'tanzania',
    '882': 'samoa',
    '762': 'tayikistan',
    '016': 'samoa americana',
    '626': 'timor oriental',
    '652': 'san bartolome',
    '768': 'togo',
    '659': 'san cristobal y nieves',
    '772': 'tokelau',
    '674': 'san marino',
    '776': 'tonga',
    '663': 'san martin',
    '780': 'trinidad y tobago',
    '666': 'san pedro y miquelon',
    '788': 'tunez',
    '670': 'san vicente y las granadinas',
    '795': 'turkmenistan',
    '654': 'santa helena, a. y t.',
    '792': 'turquia',
    '662': 'santa lucia',
    '798': 'tuvalu',
    '678': 'santo tome y principe',
    '804': 'ucrania',
    '686': 'senegal',
    '800': 'uganda',
    '688': 'serbia',
    '858': 'uruguay',
    '690': 'seychelles',
    '860': 'uzbekistan',
    '694': 'sierra leona',
    '548': 'vanuatu',
    '702': 'singapur',
    '336': 'ciudad del vaticano',
    '534': 'sint maarten',
    '862': 'venezuela',
    '760': 'siria',
    '704': 'vietnam',
    '706': 'somalia',
    '876': 'wallis y futuna',
    '144': 'sri lanka',
    '887': 'yemen',
    '748': 'suazilandia',
    '262': 'yibuti',
    '710': 'sudafrica',
    '894': 'zambia',
    '729': 'sudan',
    '716': 'zimbabue',
    'ZZZ': 'desconocido',
    '724.0': 'espana'
}

### Definición de correspondencias según Anexo solicitud RAE CMBD 2018

In [6]:
CMBD_DOMAINS = {
    'SEXO': {
        1: 'varon',
        2: 'mujer', 
        3: 'indeterminado',
        9: 'no especificado'
    },
    'TIPO_INGRESO': {
        1: 'urgente',
        2: 'programado',
        9: 'no especificado'
    },
    'TIPO_ALTA': {
        1: 'domicilio',
        2: 'traslado a otro hospital',
        3: 'alta voluntaria',
        4: 'exitus',
        5: 'traslado a centro sociosanitario',
        9: 'otros'
    }
}

### Funciones de Limpieza Estándar

In [4]:
def clean_column_names(df):
    """
    Normaliza los nombres de las columnas de un DataFrame siguiendo el estándar snake_case.
    - Convierte a minúsculas
    - Reemplaza tildes y caracteres especiales
    - Elimina artículos comunes
    - Reemplaza espacios y caracteres especiales por guiones bajos
    - Elimina guiones bajos múltiples consecutivos
    """
    new_columns = []
    
    for col in df.columns:
        # Convertir a string por si acaso
        col_str = str(col)
        
        # Reemplazar caracteres especiales y tildes
        for old, new in REPLACEMENT_MAPPING.items():
            col_str = col_str.replace(old, new)
        
        # Convertir a minúsculas
        col_str = col_str.lower()
        
        # Añadir guiones bajos al inicio y final para facilitar la eliminación de artículos
        col_str = '_' + col_str + '_'
        
        # Eliminar artículos
        for article, replacement in ARTICLES_MAPPING.items():
            col_str = col_str.replace(article, replacement)
        
        # Eliminar guiones bajos múltiples consecutivos
        while '__' in col_str:
            col_str = col_str.replace('__', '_')
        
        # Eliminar guiones bajos al inicio y al final
        col_str = col_str.strip('_')
        
        new_columns.append(col_str)
    
    # Asignar los nuevos nombres
    df.columns = new_columns
    
    return df

In [5]:
def clean_string_series(series):
    """
    Limpia una Serie de pandas normalizando texto:
    - Convierte a minúsculas
    - Reemplaza tildes y caracteres especiales
    - Elimina artículos comunes
    - Elimina espacios extras
    """
    # Convertir a minúsculas
    series = series.str.lower()
    
    # Reemplazar tildes y caracteres especiales
    for old, new in REPLACEMENT_MAPPING.items():
        series = series.str.replace(old, new, regex=False)
    
    # Añadir espacios alrededor para eliminar artículos
    series = ' ' + series + ' '
    
    # Eliminar artículos (convertir mapping de _ a espacio)
    articles_space = {k.replace('_', ' '): ' ' for k in ARTICLES_MAPPING.keys()}
    for article, replacement in articles_space.items():
        series = series.str.replace(article, replacement, regex=False)
    
    # Eliminar espacios múltiples
    series = series.str.replace(r'\s+', ' ', regex=True)
    
    # Eliminar espacios al inicio y final
    series = series.str.strip()
    
    return series

## Carga de datos y limpieza inicial

In [7]:
# Carga de datos
file_path = '../raw_data/SaludMental.xls'
df_raw = pd.read_excel(file_path)

## Estudio de los tipos de datos por columnas

In [8]:
df_raw.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21210 entries, 0 to 21209
Data columns (total 111 columns):
 #    Column                     Non-Null Count  Dtype         
---   ------                     --------------  -----         
 0    Comunidad Autónoma         21210 non-null  object        
 1    Nombre                     21210 non-null  object        
 2    Fecha de nacimiento        21210 non-null  datetime64[ns]
 3    Sexo                       21210 non-null  int64         
 4    CCAA Residencia            0 non-null      float64       
 5    Fecha de Ingreso           21210 non-null  datetime64[ns]
 6    Circunstancia de Contacto  21210 non-null  int64         
 7    Fecha de Fin Contacto      21210 non-null  object        
 8    Tipo Alta                  21210 non-null  int64         
 9    Estancia Días              21210 non-null  int64         
 10   Diagnóstico Principal      21210 non-null  object        
 11   Categoría                  21210 non-null  object   

### Normalización de los nombres de las columnas y eliminación de información sensible

In [9]:
df = clean_column_names(df_raw)
columna_nombres = df['nombre']
df = df.drop(columns=['nombre'])
df.head()


,comunidad_autonoma,fecha_nacimiento,sexo,ccaa_residencia,fecha_ingreso,circunstancia_contacto,fecha_fin_contacto,tipo_alta,estancia_dias,diagnostico_principal,categoria,diagnostico_2,diagnostico_3,diagnostico_4,diagnostico_5,diagnostico_6,diagnostico_7,diagnostico_8,diagnostico_9,diagnostico_10,diagnostico_11,diagnostico_12,diagnostico_13,diagnostico_14,fecha_intervencion,procedimiento_1,procedimiento_2,procedimiento_3,procedimiento_4,procedimiento_5,procedimiento_6,procedimiento_7,procedimiento_8,procedimiento_9,procedimiento_10,procedimiento_11,procedimiento_12,procedimiento_13,procedimiento_14,procedimiento_15,procedimiento_16,procedimiento_17,procedimiento_18,procedimiento_19,procedimiento_20,gdr_ap,cdm_ap,tipo_gdr_ap,valor_peso_espanol,grd_apr,cdm_apr,tipo_gdr_apr,valor_peso_americano_apr,nivel_severidad_apr,riesgo_mortalidad_apr,servicio,edad,reingreso,coste_apr,gdr_ir,tipo_gdr_ir,tipo_proceso_ir,cie,numero_registro_anual,centro_recodificado,cip_sns_recodificado,pais_nacimiento,pais_residencia,fecha_inicio_contacto,regimen_financiacion,procedencia,continuidad_asistencial,ingreso_en_uci,dias_uci,diagnostico_15,diagnostico_16,diagnostico_17,diagnostico_18,diagnostico_19,diagnostico_20,poa_diagnostico_principal,poa_diagnostico_2,poa_diagnostico_3,poa_diagnostico_4,poa_diagnostico_5,poa_diagnostico_6,poa_diagnostico_7,poa_diagnostico_8,poa_diagnostico_9,poa_diagnostico_10,poa_diagnostico_11,poa_diagnostico_12,poa_diagnostico_13,poa_diagnostico_14,poa_diagnostico_15,poa_diagnostico_16,poa_diagnostico_17,poa_diagnostico_18,poa_diagnostico_19,poa_diagnostico_20,procedimiento_externo_1,procedimiento_externo_2,procedimiento_externo_3,procedimiento_externo_4,procedimiento_externo_5,procedimiento_externo_6,tipo_grd_apr,peso_espanol_apr,edad_en_ingreso,mes_ingreso
0,ANDALUCÍA,1951-08-17,2,NaN,2016-01-01,1,08/01/2016,1,7,F25.0,"Esquizofrenia, trastornos esquizotípicos y tra...",Z63.79,Z91.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750,19,NaN,NaN,2,1,PSQ,64,NaN,6340,NaN,NaN,NaN,10,8537155.0,-2088791444897189888,109457269-593755146,724,724,01012016 1622,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,E,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,1.393611,64,2016-01
1,ANDALUCÍA,1929-03-20,2,NaN,2016-01-01,1,08/01/2016,1,7,F41.9,"Trastornos neuróticos, trastornos relacionados...",I11.9,I35.8,E11.9,I87.2,Z95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4B02XSZ,B246ZZZ,4A02X4Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,756,19,NaN,NaN,1,2,CAR,86,NaN,2771,NaN,NaN,NaN,10,8992115.0,-1166333372325380096,-1589750168781380096,ZZZ,724,01012016 0453,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,S,S,S,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,0.609264,86,2016-01
2,ANDALUCÍA,1976-11-25,1,NaN,2016-01-01,1,11/01/2016,1,10,F60.2,Trastornos de la personalidad y del comportami...,F19.288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,752,19,NaN,NaN,2,1,PSQ,39,NaN,4009,NaN,NaN,NaN,10,8998349.0,17490445801063320188,-5406560181117020160,724,724,01012016 1301,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,0.881297,39,2016-01
3,ANDALUCÍA,1976-11-10,2,NaN,2016-01-01,1,27/01/2016,1,26,F20.0,"Esquizofrenia, trastornos esquizotípicos y tra...",C07,F17.210,F12.20,F14.10,F10.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750,19,NaN,NaN,1,2,PSQ,39,NaN,6073,NaN,NaN,NaN,10,8800205.0,-3960068041784730112,-1823171082,724,724,01012016 1446,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,S,S,S,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

## Transformación de las diferentes columnas

### Eliminación de columnas con solo valores nulos

In [10]:
null_columns = df.columns[df.isnull().all()]

In [11]:
print("Columnas con solo valores nulos:")
print(null_columns)
df = df.drop(columns=null_columns)

Columnas con solo valores nulos:
Index(['ccaa_residencia', 'gdr_ap', 'cdm_ap', 'tipo_gdr_ap',
       'valor_peso_espanol', 'tipo_gdr_apr', 'valor_peso_americano_apr',
       'reingreso', 'gdr_ir', 'tipo_gdr_ir', 'tipo_proceso_ir',
       'procedimiento_externo_4', 'procedimiento_externo_5',
       'procedimiento_externo_6'],
      dtype='object')


### Comunidad autónoma

In [12]:
df['comunidad_autonoma'] = clean_string_series(df['comunidad_autonoma'])

### Sexo

In [13]:
df['sexo'] = df['sexo'].map(CMBD_DOMAINS['SEXO'])

### Fecha de fin de contacto

In [14]:
df['fecha_fin_contacto'] = pd.to_datetime(df['fecha_fin_contacto'], format='%d/%m/%Y', errors='coerce')

### Análisis y tratamiento de la fecha de intervención

In [15]:
print(df['fecha_intervencion'].dropna())

142      19012016 1817
149      05022016 0000
602      05022016 0845
747      10022016 1000
795      18022016 0945
             ...      
19826    31102018 1257
19830    20112018 1720
20220    07112018 2018
20633    27122018 0900
20770    06122018 0432
Name: fecha_intervencion, Length: 141, dtype: object


In [16]:
# Limpiar: hacer split por espacio y quedarse con la primera parte (posición 0)
df['fecha_intervencion'] = df['fecha_intervencion'].astype(str).str.split(' ').str[0]

# Convertir a datetime con formato DDMMYYYY
df['fecha_intervencion'] = pd.to_datetime(
    df['fecha_intervencion'], 
    format='%d%m%Y', 
    errors='coerce'
)

# Verificar el resultado
print("\n\nDespués de la conversión:")
print(f"Tipo de dato: {df['fecha_intervencion'].dtype}")
print(f"Valores nulos: {df['fecha_intervencion'].isna().sum()}")
print(f"Valores no nulos: {df['fecha_intervencion'].notna().sum()}")



Después de la conversión:
Tipo de dato: datetime64[ns]
Valores nulos: 21069
Valores no nulos: 141


### Fecha de inicio de contacto

In [17]:
# Limpiar: hacer split por espacio y quedarse con la primera parte (posición 0)
df['fecha_inicio_contacto'] = df['fecha_inicio_contacto'].astype(str).str.split(' ').str[0]

# Convertir a datetime con formato DDMMYYYY
df['fecha_inicio_contacto'] = pd.to_datetime(
    df['fecha_inicio_contacto'], 
    format='%d%m%Y', 
    errors='coerce'
)

# Verificar el resultado
print("\n\nDespués de la conversión:")
print(f"Tipo de dato: {df['fecha_inicio_contacto'].dtype}")
print(f"Valores nulos: {df['fecha_inicio_contacto'].isna().sum()}")
print(f"Valores no nulos: {df['fecha_inicio_contacto'].notna().sum()}")



Después de la conversión:
Tipo de dato: datetime64[ns]
Valores nulos: 0
Valores no nulos: 21210


### Mes de ingreso

In [18]:
df['mes_ingreso'] = df['mes_ingreso'].astype(str).str[-2:].astype(int)

# Verificar el resultado
print("\n\nDespués de la conversión:")
print(f"Tipo de dato: {df['mes_ingreso'].dtype}")
print(f"Valores nulos: {df['mes_ingreso'].isna().sum()}")



Después de la conversión:
Tipo de dato: int64
Valores nulos: 0


### Tipo de alta

In [19]:
# Aplicar el mapping usando el diccionario CMBD_DOMAINS
df['tipo_alta_desc'] = df['tipo_alta'].map(CMBD_DOMAINS['TIPO_ALTA'])

# Verificar el resultado
print("\n\nResultado del mapping:")
print(df[['tipo_alta', 'tipo_alta_desc']].value_counts())




Resultado del mapping:
tipo_alta  tipo_alta_desc                  
1          domicilio                           19425
3          alta voluntaria                       524
2          traslado a otro hospital              509
5          traslado a centro sociosanitario      368
4          exitus                                 41
9          otros                                  19
Name: count, dtype: int64


### Decodificación de diagnósticos: guardado de correspondencias

In [22]:
# Crear directorio jsons si no existe
jsons_dir = '../jsons'
os.makedirs(jsons_dir, exist_ok=True)

# Guardar el diccionario CIE-10 completo en formato JSON
json_path = os.path.join(jsons_dir, 'diagnosticos.json')

with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(CIE10_COMPLETO, f, ensure_ascii=False, indent=2)

### Decodificación de procedimientos: guardado de correspondencias

In [25]:
# Identificar todas las columnas de procedimientos
columnas_procedimientos = [col for col in df.columns if 'procedimiento' in col.lower()]
print(f"Columnas de procedimientos encontradas: {len(columnas_procedimientos)}")
print(columnas_procedimientos[:10])  # Mostrar las primeras 10

# Extraer todos los códigos únicos de procedimientos de todas las columnas
codigos_procedimientos = set()
for col in columnas_procedimientos:
    codigos_unicos = df[col].dropna().unique()
    codigos_procedimientos.update(codigos_unicos)

Columnas de procedimientos encontradas: 23
['procedimiento_1', 'procedimiento_2', 'procedimiento_3', 'procedimiento_4', 'procedimiento_5', 'procedimiento_6', 'procedimiento_7', 'procedimiento_8', 'procedimiento_9', 'procedimiento_10']


In [26]:
# Función para categorizar códigos ICD-10-PCS basándose en su estructura
def categorizar_procedimiento_icd10pcs(codigo):
    """
    Categoriza un código ICD-10-PCS basándose en su primer carácter (Sección)
    """
    if pd.isna(codigo) or len(str(codigo)) < 2:
        return 'Código inválido'
    
    codigo_str = str(codigo).strip()
    
    # Primera letra: Sección
    seccion = codigo_str[0] if len(codigo_str) > 0 else ''
    
    secciones = {
        '0': 'Médico y Quirúrgico',
        '1': 'Obstetricia',
        '2': 'Colocación',
        '3': 'Administración',
        '4': 'Medición y Monitoreo',
        '5': 'Asistencia y Rendimiento Extracorpóreo',
        '6': 'Terapias Extracorpóreas',
        '7': 'Osteopatía',
        '8': 'Otros Procedimientos',
        '9': 'Quiropráctica',
        'B': 'Imagen',
        'C': 'Medicina Nuclear',
        'D': 'Radioterapia',
        'F': 'Rehabilitación Física y Audiología Diagnóstica',
        'G': 'Salud Mental',
        'H': 'Tratamiento de Abuso de Sustancias',
        'X': 'Nuevas Tecnologías',
    }
    
    seccion_desc = secciones.get(seccion, f'Sección desconocida ({seccion})')
    
    # Si es sección 0 (Médico y Quirúrgico), detallar más
    if seccion == '0' and len(codigo_str) >= 3:
        sistema = codigo_str[1:3]
        sistemas_corporales = {
            '00': 'Sistema Nervioso Central',
            '01': 'Sistema Nervioso Periférico',
            '02': 'Corazón y Grandes Vasos',
            '03': 'Arterias Superiores',
            '04': 'Venas Superiores',
            '05': 'Venas Inferiores',
            '06': 'Arterias Inferiores',
            '07': 'Linfático y Hemático',
            '08': 'Ojo',
            '09': 'Oído, Nariz, Seno',
            '0B': 'Sistema Respiratorio',
            '0C': 'Boca y Garganta',
            '0D': 'Sistema Gastrointestinal',
            '0F': 'Sistema Hepatobiliar y Páncreas',
            '0G': 'Sistema Endocrino',
            '0H': 'Piel y Mama',
            '0J': 'Tejido Subcutáneo y Fascia',
            '0K': 'Músculos',
            '0L': 'Tendones',
            '0M': 'Bursas y Ligamentos',
            '0N': 'Cabeza y Huesos Faciales',
            '0P': 'Huesos Superiores',
            '0Q': 'Huesos Inferiores',
            '0R': 'Articulaciones Superiores',
            '0S': 'Articulaciones Inferiores',
            '0T': 'Sistema Urinario',
            '0U': 'Sistema Reproductivo Femenino',
            '0V': 'Sistema Reproductivo Masculino',
            '0W': 'Regiones Anatómicas Generales',
            '0X': 'Regiones Anatómicas Superiores',
            '0Y': 'Regiones Anatómicas Inferiores',
        }
        sistema_desc = sistemas_corporales.get(sistema, sistema)
        return f'{seccion_desc} - {sistema_desc}'
    
    # Secciones especiales de salud mental
    elif seccion == 'G':
        return 'Salud Mental'
    elif seccion == 'H':
        return 'Tratamiento de Abuso de Sustancias'
    elif seccion == 'F':
        return 'Rehabilitación y Terapia Física'
    
    return seccion_desc

In [27]:
# Crear diccionario completo con categorización
procedimientos_completo = {}

# Agregar los códigos conocidos
procedimientos_completo.update(PROCEDIMIENTOS_ICD10_PCS)

# Para códigos no mapeados, agregar con categorización automática
for codigo in sorted(codigos_procedimientos):
    codigo_str = str(codigo).strip()
    if codigo_str not in procedimientos_completo:
        categoria = categorizar_procedimiento_icd10pcs(codigo_str)
        procedimientos_completo[codigo_str] = f'{codigo_str} - {categoria}'


secciones_count = {}
for codigo in codigos_procedimientos:
    categoria = categorizar_procedimiento_icd10pcs(codigo)
    secciones_count[categoria] = secciones_count.get(categoria, 0) + 1

for categoria, count in sorted(secciones_count.items(), key=lambda x: x[1], reverse=True)[:15]:
    print(f"  {categoria}: {count} códigos")

# Guardar en JSON
jsons_dir = '../jsons'
os.makedirs(jsons_dir, exist_ok=True)
json_path = os.path.join(jsons_dir, 'procedimientos.json')

with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(procedimientos_completo, f, ensure_ascii=False, indent=2)

  Imagen: 175 códigos
  Administración: 46 códigos
  Medición y Monitoreo: 31 códigos
  Salud Mental: 23 códigos
  Médico y Quirúrgico - DB: 22 códigos
  Tratamiento de Abuso de Sustancias: 20 códigos
  Medicina Nuclear: 19 códigos
  Médico y Quirúrgico - UT: 13 códigos
  Asistencia y Rendimiento Extracorpóreo: 13 códigos
  Rehabilitación y Terapia Física: 11 códigos
  Colocación: 10 códigos
  Médico y Quirúrgico - HB: 8 códigos
  Médico y Quirúrgico - PS: 8 códigos
  Médico y Quirúrgico - T9: 8 códigos
  Médico y Quirúrgico - Oído, Nariz, Seno: 6 códigos


### Actualización de la columna edad

In [29]:
# Fecha de referencia (hoy)
fecha_actual = pd.Timestamp.now()
print(f"\nFecha de referencia para el cálculo: {fecha_actual.date()}")

# Calcular edad en años
df['edad'] = ((fecha_actual - df['fecha_nacimiento']).dt.days / 365.25).astype(int)

# Verificar el resultado
print(f"\nColumna 'edad' actualizada exitosamente")
print(f"\nEstadísticas de edad:")
print(df['edad'].describe())

# Verificar rango de edades
print(f"\nRango de edades:")
print(f"  Edad mínima: {df['edad'].min()} años")
print(f"  Edad máxima: {df['edad'].max()} años")
print(f"  Edad promedio: {df['edad'].mean():.2f} años")
print(f"  Edad mediana: {df['edad'].median():.0f} años")

# Mostrar algunos ejemplos de verificación
print(f"\nEjemplos de verificación (primeras 10 filas):")
print(df[['fecha_nacimiento', 'edad']].head(10))


Fecha de referencia para el cálculo: 2025-10-16

Columna 'edad' actualizada exitosamente

Estadísticas de edad:
count    21210.000000
mean        51.919142
std         14.135110
min          7.000000
25%         42.000000
50%         52.000000
75%         62.000000
max        104.000000
Name: edad, dtype: float64

Rango de edades:
  Edad mínima: 7 años
  Edad máxima: 104 años
  Edad promedio: 51.92 años
  Edad mediana: 52 años

Ejemplos de verificación (primeras 10 filas):
  fecha_nacimiento  edad
0       1951-08-17    74
1       1929-03-20    96
2       1976-11-25    48
3       1976-11-10    48
4       1977-04-28    48
5       1986-01-19    39
6       1995-10-06    30
7       1964-04-22    61
8       1966-10-06    59
9       1987-01-22    38


### Decodificación de países (de nacimiento y residencia)

In [31]:
df['pais_nacimiento'] = df['pais_nacimiento'].astype(str).map(PAISES_NUMERICOS).fillna(df['pais_nacimiento'])
df['pais_residencia'] = df['pais_residencia'].astype(str).map(PAISES_NUMERICOS).fillna(df['pais_residencia'])

## Exportación de datos limpios

In [32]:
# Exportar el dataframe df a csv a ../cleaned_data
cleaned_dir = '../cleaned_data'
os.makedirs(cleaned_dir, exist_ok=True)
csv_path = os.path.join(cleaned_dir, 'salud_mental_cleaned.csv')
df.to_csv(csv_path, index=False)